# Feature Selection and Testing Framework

This notebook provides a structured approach to:
1. Easily enable/disable different feature groups
2. Test multiple feature combinations systematically
3. Compare performance across different configurations


In [1]:
# ===== IMPORTS =====
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from itertools import combinations, product

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# ===== CONFIGURATION =====
# Define random seed
seed = 42

# Individual Feature Configuration - Set to True/False to enable/disable each feature
FEATURE_CONFIG = {
    # === ATTRIBUTE FEATURES ===
    'attribute_i': True,              # Individual attribute value for node i
    'attribute_j': True,              # Individual attribute value for node j
    'attribute_equality': True,       # Attribute equality indicator
    
    # === BASIC NODE FEATURES ===
    'degree_i': True,                 # Degree of node i
    'degree_j': True,                 # Degree of node j
    'clustering_i': True,             # Clustering coefficient of node i
    'clustering_j': True,             # Clustering coefficient of node j
    'pagerank_i': True,               # PageRank of node i
    'pagerank_j': True,               # PageRank of node j
    
    # === NEIGHBORHOOD FEATURES ===
    'common_neighbors': True,         # Number of common neighbors
    'jaccard_coefficient': True,      # Jaccard coefficient
    'adamic_adar': True,              # Adamic-Adar index
    'resource_allocation': True,       # Resource Allocation index
    'preferential_attachment': True,   # Preferential Attachment
    'salton_index': True,             # Salton index
    'sorensen_index': True,           # Sorensen index
    'two_hop_neighbors': True,        # 2-hop neighbors count
    
    # === DERIVED FEATURES ===
    'degree_sum': True,               # Sum of degrees
    'degree_diff': True,              # Absolute difference of degrees
    'degree_product': True,            # Product of degrees
    'degree_ratio': True,             # Ratio of degrees (min/max)
    'clustering_avg': True,           # Average clustering coefficient
    'pagerank_avg': True,             # Average PageRank
    
    # === COMMUNITY FEATURES (Resolution 1.0) ===
    'same_community': True,           # Same community indicator
    'community_size_i': True,         # Community size of node i
    'community_size_j': True,         # Community size of node j
    'community_size_diff': True,      # Absolute difference in community sizes
    'community_size_ratio': True,     # Ratio of community sizes
    
    # === COMMUNITY FEATURES (Resolution 0.5 - Bigger Communities) ===
    'same_community_big': True,       # Same community indicator (big communities)
    'community_size_i_big': True,     # Community size of node i (big communities)
    'community_size_j_big': True,     # Community size of node j (big communities)
    'community_size_diff_big': True,  # Absolute difference in community sizes (big communities)
    'community_size_ratio_big': True, # Ratio of community sizes (big communities)
}

# Model Configuration
MODEL_CONFIG = {
    'n_estimators': 100,
    'max_depth': 3,
    'min_samples_split': 10,
    'min_samples_leaf': 3,
    'random_state': seed,
    'n_jobs': -1
}

# Cross-validation Configuration
CV_CONFIG = {
    'n_splits': 5,
    'shuffle': True,
    'random_state': seed
}

# Community Configuration
COMMUNITY_CONFIG = {
    'resolution_small': 1.0,  # Smaller communities (more granular)
    'resolution_big': 0.5,    # Bigger communities (less granular)
    'test_resolutions': np.arange(0.3, 2.0, 0.1).tolist()  # For testing multiple resolutions
}

print("✓ Configuration loaded")


✓ Configuration loaded


In [2]:
# ===== DATA LOADING =====
def load_data():
    """Load graph and attributes data"""
    # Load graph
    path = "./../assignment2_files_2025/edges_train.edgelist"
    G = nx.read_edgelist(path, delimiter=',', nodetype=int, create_using=nx.Graph())
    print(f"Graph loaded with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    
    # Load attributes
    pathattributes = "./../assignment2_files_2025/attributes.csv"
    attributes_df = pd.read_csv(pathattributes)
    node_id_col = attributes_df.columns[0]
    attribute_cols = [col for col in attributes_df.columns if col != node_id_col]
    
    # Encode categorical to numerical
    for col in attribute_cols:
        if attributes_df[col].dtype == 'object':
            le = LabelEncoder()
            attributes_df[col] = le.fit_transform(attributes_df[col].astype(str))
            print(f"   ✓ Encoded '{col}': {list(le.classes_)}")
    
    attributes_dict = attributes_df.set_index(node_id_col).to_dict('index')
    
    return G, attributes_dict, attribute_cols

# Load data
G, attributes_dict, attribute_cols = load_data()
print("✓ Data loaded successfully")


Graph loaded with 1500 nodes and 6600 edges
   ✓ Encoded 'attribute': ['d', 'f', 'l', 'm', 'x', 'y']
✓ Data loaded successfully


In [3]:
# ===== FEATURE ENGINEERING =====
def get_features(G, i, j, feature_config=FEATURE_CONFIG, precomputed_metrics=None):
    """
    Features voor node pair (i, j) based on configuration
    Args:
        G: NetworkX graph
        i, j: Node pair
        feature_config: Feature configuration dictionary
        precomputed_metrics: Dictionary with precomputed metrics (pa, pagerank, clustering, community_dict, comm_sizes)
    Returns: (feature_array, feature_names)
    """
    features = []
    feature_names = []
    
    # Get attribute values
    attrs_i = attributes_dict.get(i, {})
    attrs_j = attributes_dict.get(j, {})
    
    # --- Attribute features ---
    for col in attribute_cols:
        val_i = attrs_i.get(col, 0)
        val_j = attrs_j.get(col, 0)
        
        if feature_config['attribute_i']:
            features.append(val_i)
            feature_names.append(f"{col}_i")
        
        if feature_config['attribute_j']:
            features.append(val_j)
            feature_names.append(f"{col}_j")
        
        if feature_config['attribute_equality']:
            features.append(int(val_i == val_j))
            feature_names.append(f"{col}_eq")
    
    # --- Basic node features ---
    deg_i = G.degree(i)
    deg_j = G.degree(j)
    
    # Use precomputed metrics if available, otherwise compute on-the-fly
    if precomputed_metrics:
        cc_i = precomputed_metrics['clustering'].get(i, 0)
        cc_j = precomputed_metrics['clustering'].get(j, 0)
        pr_i = precomputed_metrics['pagerank'].get(i, 0)
        pr_j = precomputed_metrics['pagerank'].get(j, 0)
    else:
        # Fallback: compute on-the-fly (not recommended for performance)
        cc_i = nx.clustering(G, i)
        cc_j = nx.clustering(G, j)
        pr_i = nx.pagerank(G)[i] if i in nx.pagerank(G) else 0
        pr_j = nx.pagerank(G)[j] if j in nx.pagerank(G) else 0
    
    if feature_config['degree_i']:
        features.append(deg_i)
        feature_names.append('deg_i')
    
    if feature_config['degree_j']:
        features.append(deg_j)
        feature_names.append('deg_j')
    
    if feature_config['clustering_i']:
        features.append(cc_i)
        feature_names.append('cc_i')
    
    if feature_config['clustering_j']:
        features.append(cc_j)
        feature_names.append('cc_j')
    
    if feature_config['pagerank_i']:
        features.append(pr_i)
        feature_names.append('pr_i')
    
    if feature_config['pagerank_j']:
        features.append(pr_j)
        feature_names.append('pr_j')
    
    # --- Neighborhood features ---
    common = list(nx.common_neighbors(G, i, j))
    cn_ij = len(common)
    
    if feature_config['common_neighbors']:
        features.append(cn_ij)
        feature_names.append('cn_ij')
    
    if feature_config['jaccard_coefficient']:
        jc_ij = next(nx.jaccard_coefficient(G, [(i,j)]))[2]
        features.append(jc_ij)
        feature_names.append('jc_ij')
    
    if feature_config['adamic_adar']:
        aa_ij = sum(1.0 / np.log(G.degree(z)) for z in common if G.degree(z) > 1)
        features.append(aa_ij)
        feature_names.append('aa_ij')
    
    if feature_config['resource_allocation']:
        ra_ij = sum(1.0 / G.degree(z) for z in common if G.degree(z) > 0)
        features.append(ra_ij)
        feature_names.append('ra_ij')
    
    if feature_config['preferential_attachment']:
        if precomputed_metrics and 'pa' in precomputed_metrics:
            pa_ij = precomputed_metrics['pa'][i, j]
        else:
            # Fallback: compute on-the-fly (very expensive!)
            pa_ij = next(nx.preferential_attachment(G, [(i, j)]))[2]
        features.append(pa_ij)
        feature_names.append('pa_ij')
    
    if feature_config['salton_index']:
        salton = cn_ij / np.sqrt(deg_i * deg_j) if (deg_i * deg_j) > 0 else 0
        features.append(salton)
        feature_names.append('salton')
    
    if feature_config['sorensen_index']:
        sorensen = (2 * cn_ij) / (deg_i + deg_j) if (deg_i + deg_j) > 0 else 0
        features.append(sorensen)
        feature_names.append('sorensen')
    
    if feature_config['two_hop_neighbors']:
        # 2 hop neighbors
        dist2_i = set(nx.single_source_shortest_path_length(G, i, cutoff=2))
        dist2_j = set(nx.single_source_shortest_path_length(G, j, cutoff=2))
        
        # remove the nodes themselves and their direct neighbors
        dist2_i -= {i} | set(G.neighbors(i))
        dist2_j -= {j} | set(G.neighbors(j))
        
        # intersection of these "exactly distance-2" neighborhoods
        dist2_count = len(dist2_i & dist2_j)
        features.append(dist2_count)
        feature_names.append('dist2_count')
    
    # --- Derived features ---
    if feature_config['degree_sum']:
        deg_sum = deg_i + deg_j
        features.append(deg_sum)
        feature_names.append('deg_sum')
    
    if feature_config['degree_diff']:
        deg_diff = abs(deg_i - deg_j)
        features.append(deg_diff)
        feature_names.append('deg_diff')
    
    if feature_config['degree_product']:
        deg_product = deg_i * deg_j
        features.append(deg_product)
        feature_names.append('deg_product')
    
    if feature_config['degree_ratio']:
        deg_ratio = min(deg_i, deg_j) / max(deg_i, deg_j) if max(deg_i, deg_j) > 0 else 0
        features.append(deg_ratio)
        feature_names.append('deg_ratio')
    
    if feature_config['clustering_avg']:
        cc_avg = (cc_i + cc_j) / 2
        features.append(cc_avg)
        feature_names.append('cc_avg')
    
    if feature_config['pagerank_avg']:
        pr_avg = (pr_i + pr_j) / 2
        features.append(pr_avg)
        feature_names.append('pr_avg')
    
    # --- Community features (Small communities) ---
    if precomputed_metrics and 'community_dict_small' in precomputed_metrics:
        comm_i_small = precomputed_metrics['community_dict_small'].get(i, -1)
        comm_j_small = precomputed_metrics['community_dict_small'].get(j, -1)
        comm_sizes_small = precomputed_metrics['comm_sizes_small']
    else:
        # Fallback: compute communities on-the-fly (expensive!)
        communities_small = list(nx.algorithms.community.greedy_modularity_communities(G, resolution=COMMUNITY_CONFIG['resolution_small']))
        community_dict_small = {n: cid for cid, nodes in enumerate(communities_small) for n in nodes}
        comm_sizes_small = {cid: len(nodes) for cid, nodes in enumerate(communities_small)}
        comm_i_small = community_dict_small.get(i, -1)
        comm_j_small = community_dict_small.get(j, -1)
    
    if feature_config['same_community']:
        same_comm_small = int(comm_i_small == comm_j_small)
        features.append(same_comm_small)
        feature_names.append('same_comm')
    
    if feature_config['community_size_i']:
        size_i_small = comm_sizes_small.get(comm_i_small, 0)
        features.append(size_i_small)
        feature_names.append('size_i')
    
    if feature_config['community_size_j']:
        size_j_small = comm_sizes_small.get(comm_j_small, 0)
        features.append(size_j_small)
        feature_names.append('size_j')
    
    if feature_config['community_size_diff']:
        size_i_small = comm_sizes_small.get(comm_i_small, 0)
        size_j_small = comm_sizes_small.get(comm_j_small, 0)
        abs_size_diff_small = abs(size_i_small - size_j_small)
        features.append(abs_size_diff_small)
        feature_names.append('abs_size_diff')
    
    if feature_config['community_size_ratio']:
        size_i_small = comm_sizes_small.get(comm_i_small, 0)
        size_j_small = comm_sizes_small.get(comm_j_small, 0)
        if size_i_small + size_j_small > 0:
            rel_size_ratio_small = size_i_small / (size_j_small + 1e-6)
        else:
            rel_size_ratio_small = 0
        features.append(rel_size_ratio_small)
        feature_names.append('rel_size_ratio')
    
    # --- Community features (Big communities) ---
    if precomputed_metrics and 'community_dict_big' in precomputed_metrics:
        comm_i_big = precomputed_metrics['community_dict_big'].get(i, -1)
        comm_j_big = precomputed_metrics['community_dict_big'].get(j, -1)
        comm_sizes_big = precomputed_metrics['comm_sizes_big']
    else:
        # Fallback: compute communities on-the-fly (expensive!)
        communities_big = list(nx.algorithms.community.greedy_modularity_communities(G, resolution=COMMUNITY_CONFIG['resolution_big']))
        community_dict_big = {n: cid for cid, nodes in enumerate(communities_big) for n in nodes}
        comm_sizes_big = {cid: len(nodes) for cid, nodes in enumerate(communities_big)}
        comm_i_big = community_dict_big.get(i, -1)
        comm_j_big = community_dict_big.get(j, -1)
    
    if feature_config['same_community_big']:
        same_comm_big = int(comm_i_big == comm_j_big)
        features.append(same_comm_big)
        feature_names.append('same_comm_big')
    
    if feature_config['community_size_i_big']:
        size_i_big = comm_sizes_big.get(comm_i_big, 0)
        features.append(size_i_big)
        feature_names.append('size_i_big')
    
    if feature_config['community_size_j_big']:
        size_j_big = comm_sizes_big.get(comm_j_big, 0)
        features.append(size_j_big)
        feature_names.append('size_j_big')
    
    if feature_config['community_size_diff_big']:
        size_i_big = comm_sizes_big.get(comm_i_big, 0)
        size_j_big = comm_sizes_big.get(comm_j_big, 0)
        abs_size_diff_big = abs(size_i_big - size_j_big)
        features.append(abs_size_diff_big)
        feature_names.append('abs_size_diff_big')
    
    if feature_config['community_size_ratio_big']:
        size_i_big = comm_sizes_big.get(comm_i_big, 0)
        size_j_big = comm_sizes_big.get(comm_j_big, 0)
        if size_i_big + size_j_big > 0:
            rel_size_ratio_big = size_i_big / (size_j_big + 1e-6)
        else:
            rel_size_ratio_big = 0
        features.append(rel_size_ratio_big)
        feature_names.append('rel_size_ratio_big')
    
    return np.array(features, dtype=float), feature_names

def precompute_metrics(G_train, community_resolution_small=1.0, community_resolution_big=0.5):
    """
    Pre-compute all expensive graph metrics using ONLY training graph
    Args:
        G_train: NetworkX training graph (only training edges)
        community_resolution_small: Resolution parameter for small communities
        community_resolution_big: Resolution parameter for big communities
    Returns:
        Dictionary with precomputed metrics
    """
    N = G_train.number_of_nodes()
    
    # Preferential Attachment
    pa = np.zeros((N, N))
    for u, v, p in nx.preferential_attachment(G_train, [(i, j) for i in range(N) for j in range(N)]):
        pa[u, v] = p
    
    # PageRank
    pagerank = nx.pagerank(G_train)
    
    # Clustering        
    clustering = nx.clustering(G_train)
    
    # Small Communities (more granular)
    communities_small = list(nx.algorithms.community.greedy_modularity_communities(G_train, resolution=community_resolution_small))
    community_dict_small = {n: cid for cid, nodes in enumerate(communities_small) for n in nodes}
    comm_sizes_small = {cid: len(nodes) for cid, nodes in enumerate(communities_small)}
    
    # Big Communities (less granular)
    communities_big = list(nx.algorithms.community.greedy_modularity_communities(G_train, resolution=community_resolution_big))
    community_dict_big = {n: cid for cid, nodes in enumerate(communities_big) for n in nodes}
    comm_sizes_big = {cid: len(nodes) for cid, nodes in enumerate(communities_big)}
    
    return {
        'pa': pa,
        'pagerank': pagerank,
        'clustering': clustering,
        'community_dict_small': community_dict_small,
        'comm_sizes_small': comm_sizes_small,
        'community_dict_big': community_dict_big,
        'comm_sizes_big': comm_sizes_big
    }

print("✓ Feature engineering function defined")
print("✓ Precomputed metrics helper function defined")


✓ Feature engineering function defined
✓ Precomputed metrics helper function defined


In [4]:
# ===== SINGLE CONFIGURATION TESTING =====
def test_single_configuration(feature_config, model_config=MODEL_CONFIG, cv_config=CV_CONFIG, 
                            community_resolution_small=1.0, community_resolution_big=0.5):
    """Test a single feature configuration"""
    print(f"\n=== Testing Configuration ===")
    print(f"Enabled features: {[k for k, v in feature_config.items() if v]}")
    
    # Initialize results
    auc_scores = []
    acc_scores = []
    
    # K-fold cross validation
    edges = np.array(list(G.edges()))
    kf = KFold(n_splits=cv_config['n_splits'], shuffle=cv_config['shuffle'], random_state=cv_config['random_state'])
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(edges), 1):
        print(f"\n--- Fold {fold} ---")
        
        # Split edges
        train_edges = edges[train_idx]
        val_edges = edges[val_idx]
        
        # Build training graph
        G_train = nx.Graph()
        G_train.add_nodes_from(G.nodes())
        G_train.add_edges_from(train_edges)
        
        # Pre-compute metrics using helper function
        precomputed_metrics = precompute_metrics(G_train, community_resolution_small, community_resolution_big)
        
        # Build training data
        pos_train = train_edges
        rng = np.random.default_rng(seed)
        non_edges = np.array(list(nx.non_edges(G_train)))
        neg_train = non_edges[rng.choice(len(non_edges), size=len(pos_train), replace=False)]
        
        X_train_features = []
        for (u, v) in np.vstack([pos_train, neg_train]):
            feat, _ = get_features(G_train, u, v, feature_config, precomputed_metrics)
            X_train_features.append(feat)
        
        y_train = [1]*len(pos_train) + [0]*len(neg_train)
    
        # Build validation data
        pos_val = val_edges
        rng2 = np.random.default_rng(seed+1)
        non_edges = np.array(list(nx.non_edges(G_train)))
        neg_val = non_edges[rng2.choice(len(non_edges), size=len(pos_val), replace=False)]
        
        X_val_features = []
        for (u, v) in np.vstack([pos_val, neg_val]):
            feat, _ = get_features(G_train, u, v, feature_config, precomputed_metrics)
            X_val_features.append(feat)
        
        y_val = [1]*len(pos_val) + [0]*len(neg_val)
    
        # Train model
        clf = RandomForestClassifier(**model_config)
        clf.fit(X_train_features, y_train)
        
        # Evaluate
        y_pred_proba = clf.predict_proba(X_val_features)[:,1]
        auc = roc_auc_score(y_val, y_pred_proba)
        auc_scores.append(auc)
        
        y_pred_val = clf.predict(X_val_features)
        val_acc = accuracy_score(y_val, y_pred_val)
        acc_scores.append(val_acc)
        
        print(f"Fold {fold} AUC: {auc:.4f}, Accuracy: {val_acc:.4f}")
    
    # Calculate final results
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)
    mean_acc = np.mean(acc_scores)
    std_acc = np.std(acc_scores)
    
    print(f"\n=== Results ===")
    print(f"Mean AUC: {mean_auc:.4f} ± {std_auc:.4f}")
    print(f"Mean Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
    
    return {
        'mean_auc': mean_auc,
        'std_auc': std_auc,
        'mean_acc': mean_acc,
        'std_acc': std_acc,
        'auc_scores': auc_scores,
        'acc_scores': acc_scores
    }

print("✓ Single configuration testing function defined")


✓ Single configuration testing function defined


In [11]:
# ===== INDIVIDUAL FEATURE TESTING =====
def test_individual_features(max_features=10):
    """Test individual features to identify the most important ones"""
    results = []
    
    # Test each individual feature
    print("\n=== Testing Individual Features ===")
    for feature_name in FEATURE_CONFIG.keys():
        config = {key: False for key in FEATURE_CONFIG.keys()}
        config[feature_name] = True
        
        print(f"\nTesting {feature_name}...")
        result = test_single_configuration(config)
        result['config_name'] = feature_name
        result['features'] = [feature_name]
        results.append(result)
    
    # Sort by AUC to find best individual features
    results.sort(key=lambda x: x['mean_auc'], reverse=True)
    
    # Test combinations of top features
    print("\n=== Testing Top Feature Combinations ===")
    top_features = [r['config_name'] for r in results[:max_features]]
    
    # Test all combinations of 2 top features
    for combo in combinations(top_features, 2):
        config = {key: False for key in FEATURE_CONFIG.keys()}
        config_name = f"{combo[0]} + {combo[1]}"
        
        for feature in combo:
            config[feature] = True
        
        print(f"\nTesting {config_name}...")
        result = test_single_configuration(config)
        result['config_name'] = config_name
        result['features'] = list(combo)
        results.append(result)
    
    # Test all features
    print("\n=== Testing All Features ===")
    config = {key: True for key in FEATURE_CONFIG.keys()}
    result = test_single_configuration(config)
    result['config_name'] = 'all_features'
    result['features'] = list(FEATURE_CONFIG.keys())
    results.append(result)
    
    return results

# ===== FEATURE GROUP TESTING =====
def test_feature_groups():
    """Test logical feature groups for systematic analysis"""
    # Define logical feature groups
    feature_groups = {
        'attributes': ['attribute_i', 'attribute_j', 'attribute_equality'],
        'basic_node': ['degree_i', 'degree_j', 'clustering_i', 'clustering_j', 'pagerank_i', 'pagerank_j'],
        'neighborhood': ['common_neighbors', 'jaccard_coefficient', 'adamic_adar', 
                        'resource_allocation', 'preferential_attachment', 
                        'salton_index', 'sorensen_index', 'two_hop_neighbors'],
        'derived': ['degree_sum', 'degree_diff', 'degree_product', 'degree_ratio', 
                   'clustering_avg', 'pagerank_avg'],
        'community': ['same_community', 'community_size_i', 'community_size_j', 
                     'community_size_diff', 'community_size_ratio']
    }
    
    results = []
    
    # Test individual groups
    print("\n=== Testing Feature Groups ===")
    for group_name, features in feature_groups.items():
        config = {key: False for key in FEATURE_CONFIG.keys()}
        for feature in features:
            config[feature] = True
        
        print(f"\nTesting {group_name} group...")
        result = test_single_configuration(config)
        result['config_name'] = group_name
        result['features'] = features
        results.append(result)
    
    # Test some combinations
    print("\n=== Testing Group Combinations ===")
    group_names = list(feature_groups.keys())
    
    # Test all combinations of 2 groups
    for combo in combinations(group_names, 2):
        config = {key: False for key in FEATURE_CONFIG.keys()}
        config_name = f"{' + '.join(combo)}"
        
        for group in combo:
            for feature in feature_groups[group]:
                config[feature] = True
        
        print(f"\nTesting {config_name}...")
        result = test_single_configuration(config)
        result['config_name'] = config_name
        result['features'] = [f for group in combo for f in feature_groups[group]]
        results.append(result)
    
    return results

# ===== COMMUNITY RESOLUTION TESTING =====
def test_community_resolutions(feature_config=None, resolution_pairs=None):
    """
    Test different combinations of community resolutions
    Args:
        feature_config: Feature configuration (if None, uses only community features)
        resolution_pairs: List of (small_resolution, big_resolution) tuples to test
    """
    if feature_config is None:
        # Default: only community features
        feature_config = {key: False for key in FEATURE_CONFIG.keys()}
        # Enable all community features
        community_features = [k for k in FEATURE_CONFIG.keys() if 'community' in k]
        for feat in community_features:
            feature_config[feat] = True
    
    if resolution_pairs is None:
        # Default resolution pairs to test
        resolution_pairs = [
            (1.2, 1.0),   # Medium+ big
            (1.4, 1.0),   # Medium big
            (1.6, 1.0),   # Avg big
            (1.8, 1.0),   # Smaller big
            (2.0, 1.0),   # Small big
            (2.2, 1.0),   # Smallest big
        ]
    
    results = []
    
    print(f"\n=== Testing Community Resolution Combinations ===")
    print(f"Feature config: {[k for k, v in feature_config.items() if v]}")
    
    for small_res, big_res in resolution_pairs:
        print(f"\n--- Testing resolutions: small={small_res}, big={big_res} ---")
        
        result = test_single_configuration(
            feature_config, 
            community_resolution_small=small_res, 
            community_resolution_big=big_res
        )
        
        result['config_name'] = f"small_{small_res}_big_{big_res}"
        result['small_resolution'] = small_res
        result['big_resolution'] = big_res
        results.append(result)
    
    return results

# ===== COMPARE SMALL VS BIG COMMUNITY FEATURES =====
def compare_community_features():
    """Compare small community features vs big community features"""
    results = []
    
    # Test 1: Only small community features
    small_config = {key: False for key in FEATURE_CONFIG.keys()}
    small_features = [k for k in FEATURE_CONFIG.keys() if 'community' in k and 'big' not in k]
    for feat in small_features:
        small_config[feat] = True
    
    print("\n=== Testing Small Community Features Only ===")
    result = test_single_configuration(small_config)
    result['config_name'] = 'small_communities_only'
    result['features'] = small_features
    results.append(result)
    
    # Test 2: Only big community features
    big_config = {key: False for key in FEATURE_CONFIG.keys()}
    big_features = [k for k in FEATURE_CONFIG.keys() if 'community' in k and 'big' in k]
    for feat in big_features:
        big_config[feat] = True
    
    print("\n=== Testing Big Community Features Only ===")
    result = test_single_configuration(big_config)
    result['config_name'] = 'big_communities_only'
    result['features'] = big_features
    results.append(result)
    
    # Test 3: Both small and big community features
    both_config = {key: False for key in FEATURE_CONFIG.keys()}
    all_community_features = [k for k in FEATURE_CONFIG.keys() if 'community' in k]
    for feat in all_community_features:
        both_config[feat] = True
    
    print("\n=== Testing Both Small and Big Community Features ===")
    result = test_single_configuration(both_config)
    result['config_name'] = 'both_communities'
    result['features'] = all_community_features
    results.append(result)
    
    return results

print("✓ Individual feature testing functions defined")
print("✓ Community resolution testing function defined")
print("✓ Community feature comparison function defined")


✓ Individual feature testing functions defined
✓ Community resolution testing function defined
✓ Community feature comparison function defined


In [6]:
# ===== RESULTS ANALYSIS =====
def analyze_results(results):
    """Analyze and visualize results from feature testing"""
    # Create results DataFrame
    df_results = pd.DataFrame([
        {
            'config_name': r['config_name'],
            'mean_auc': r['mean_auc'],
            'std_auc': r['std_auc'],
            'mean_acc': r['mean_acc'],
            'std_acc': r['std_acc'],
            'num_features': len(r['features'])
        }
        for r in results
    ])
    
    # Sort by AUC
    df_results = df_results.sort_values('mean_auc', ascending=False)
    
    print("\n=== RESULTS SUMMARY ===")
    print(df_results.to_string(index=False))
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # AUC plot
    x_pos = range(len(df_results))
    ax1.bar(x_pos, df_results['mean_auc'], yerr=df_results['std_auc'], 
            capsize=5, alpha=0.7, color='skyblue')
    ax1.set_xlabel('Configuration')
    ax1.set_ylabel('AUC Score')
    ax1.set_title('AUC Scores by Feature Configuration')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(df_results['config_name'], rotation=45, ha='right')
    ax1.grid(True, alpha=0.3)
    
    # Accuracy plot
    ax2.bar(x_pos, df_results['mean_acc'], yerr=df_results['std_acc'], 
            capsize=5, alpha=0.7, color='lightcoral')
    ax2.set_xlabel('Configuration')
    ax2.set_ylabel('Accuracy Score')
    ax2.set_title('Accuracy Scores by Feature Configuration')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(df_results['config_name'], rotation=45, ha='right')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Feature count vs performance
    plt.figure(figsize=(10, 6))
    plt.scatter(df_results['num_features'], df_results['mean_auc'], 
                s=100, alpha=0.7, c='blue', label='AUC')
    plt.scatter(df_results['num_features'], df_results['mean_acc'], 
                s=100, alpha=0.7, c='red', label='Accuracy')
    plt.xlabel('Number of Features')
    plt.ylabel('Score')
    plt.title('Performance vs Number of Features')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return df_results

print("✓ Results analysis function defined")


✓ Results analysis function defined


## Usage Examples

### 1. Test Current Configuration
Run this to test the current individual feature configuration defined above.


In [7]:
# Test current configuration
current_result = test_single_configuration(FEATURE_CONFIG)
print(f"\nCurrent configuration performance:")
print(f"AUC: {current_result['mean_auc']:.4f} ± {current_result['std_auc']:.4f}")
print(f"Accuracy: {current_result['mean_acc']:.4f} ± {current_result['std_acc']:.4f}")



=== Testing Configuration ===
Enabled features: ['attribute_i', 'attribute_j', 'attribute_equality', 'degree_i', 'degree_j', 'clustering_i', 'clustering_j', 'pagerank_i', 'pagerank_j', 'common_neighbors', 'jaccard_coefficient', 'adamic_adar', 'resource_allocation', 'preferential_attachment', 'salton_index', 'sorensen_index', 'two_hop_neighbors', 'degree_sum', 'degree_diff', 'degree_product', 'degree_ratio', 'clustering_avg', 'pagerank_avg', 'same_community', 'community_size_i', 'community_size_j', 'community_size_diff', 'community_size_ratio', 'same_community_big', 'community_size_i_big', 'community_size_j_big', 'community_size_diff_big', 'community_size_ratio_big']

--- Fold 1 ---
Fold 1 AUC: 0.9269, Accuracy: 0.8833

--- Fold 2 ---
Fold 2 AUC: 0.9304, Accuracy: 0.8814

--- Fold 3 ---
Fold 3 AUC: 0.9309, Accuracy: 0.8867

--- Fold 4 ---
Fold 4 AUC: 0.9293, Accuracy: 0.8852

--- Fold 5 ---
Fold 5 AUC: 0.9317, Accuracy: 0.8845

=== Results ===
Mean AUC: 0.9299 ± 0.0017
Mean Accuracy: 0

### 2. Test Individual Features
Run this to test each feature individually and find the most important ones.


In [8]:
# Test individual features to find the most important ones
# results = test_individual_features(max_features=8)
# df_results = analyze_results(results)


### 3. Test Feature Groups
Run this to test logical feature groups and their combinations.


In [9]:
# Test feature groups and their combinations
# group_results = test_feature_groups()
# df_group_results = analyze_results(group_results)


### 4. Custom Individual Feature Configuration
Modify the configuration below to test specific individual features.


In [10]:
# Custom configuration - modify individual features as needed
custom_config = {
    # === ATTRIBUTE FEATURES ===
    'attribute_i': False,              # Individual attribute value for node i
    'attribute_j': False,               # Individual attribute value for node j
    'attribute_equality': True,       # Disable attribute equality
    
    # === BASIC NODE FEATURES ===
    'degree_i': False,                  # Degree of node i
    'degree_j': False,                  # Degree of node j
    'clustering_i': False,             # Disable clustering for node i
    'clustering_j': False,             # Disable clustering for node j
    'pagerank_i': False,                # PageRank of node i
    'pagerank_j': False,                # PageRank of node j
    
    # === NEIGHBORHOOD FEATURES ===
    'common_neighbors': False,          # Number of common neighbors
    'jaccard_coefficient': False,       # Jaccard coefficient
    'adamic_adar': False,              # Disable Adamic-Adar
    'resource_allocation': False,        # Resource Allocation index
    'preferential_attachment': False,   # Preferential Attachment
    'salton_index': False,             # Disable Salton index
    'sorensen_index': False,             # Sorensen index
    'two_hop_neighbors': True,        # Disable 2-hop neighbors
    
    # === DERIVED FEATURES ===
    'degree_sum': False,                # Sum of degrees
    'degree_diff': True,               # Absolute difference of degrees
    'degree_product': False,             # Product of degrees
    'degree_ratio': False,              # Ratio of degrees
    'clustering_avg': False,           # Disable average clustering
    'pagerank_avg': False,             # Disable average PageRank
    
    # === COMMUNITY FEATURES ===
    'same_community': True,             # Same community indicator
    'community_size_i': False,           # Community size of node i
    'community_size_j': False,           # Community size of node j
    'community_size_diff': False,        # Absolute difference in community sizes
    'community_size_ratio': False,     # Disable community size ratio

    # === COMMUNITY FEATURES (Resolution 0.5 - Bigger Communities) ===
    'same_community_big': True,       # Same community indicator (big communities)
    'community_size_i_big': False,     # Community size of node i (big communities)
    'community_size_j_big': False,     # Community size of node j (big communities)
    'community_size_diff_big': False,  # Absolute difference in community sizes (big communities)
    'community_size_ratio_big': False, # Ratio of community sizes (big communities)
}

# Test custom configuration
custom_result = test_single_configuration(custom_config)
print(f"\nCustom configuration performance:")
print(f"AUC: {custom_result['mean_auc']:.4f} ± {custom_result['std_auc']:.4f}")
print(f"Accuracy: {custom_result['mean_acc']:.4f} ± {custom_result['std_acc']:.4f}")



=== Testing Configuration ===
Enabled features: ['attribute_equality', 'two_hop_neighbors', 'degree_diff', 'same_community']

--- Fold 1 ---


KeyError: 'same_community_big'

In [ ]:
# Custom configuration - modify individual features as needed
custom_config = {
    # === ATTRIBUTE FEATURES ===
    'attribute_i': False,              # Individual attribute value for node i
    'attribute_j': False,               # Individual attribute value for node j
    'attribute_equality': True,       # Disable attribute equality
    
    # === BASIC NODE FEATURES ===
    'degree_i': False,                  # Degree of node i
    'degree_j': False,                  # Degree of node j
    'clustering_i': False,             # Disable clustering for node i
    'clustering_j': False,             # Disable clustering for node j
    'pagerank_i': False,                # PageRank of node i
    'pagerank_j': False,                # PageRank of node j
    
    # === NEIGHBORHOOD FEATURES ===
    'common_neighbors': False,          # Number of common neighbors
    'jaccard_coefficient': False,       # Jaccard coefficient
    'adamic_adar': False,              # Disable Adamic-Adar
    'resource_allocation': False,        # Resource Allocation index
    'preferential_attachment': False,   # Preferential Attachment
    'salton_index': False,             # Disable Salton index
    'sorensen_index': False,             # Sorensen index
    'two_hop_neighbors': True,        # Disable 2-hop neighbors
    
    # === DERIVED FEATURES ===
    'degree_sum': False,                # Sum of degrees
    'degree_diff': True,               # Absolute difference of degrees
    'degree_product': False,             # Product of degrees
    'degree_ratio': False,              # Ratio of degrees
    'clustering_avg': False,           # Disable average clustering
    'pagerank_avg': False,             # Disable average PageRank
    
    # === COMMUNITY FEATURES ===
    'same_community': True,             # Same community indicator
    'community_size_i': False,           # Community size of node i
    'community_size_j': False,           # Community size of node j
    'community_size_diff': False,        # Absolute difference in community sizes
    'community_size_ratio': True,     # Disable community size ratio
}

# Test custom configuration
custom_result = test_single_configuration(custom_config)
print(f"\nCustom configuration performance:")
print(f"AUC: {custom_result['mean_auc']:.4f} ± {custom_result['std_auc']:.4f}")
print(f"Accuracy: {custom_result['mean_acc']:.4f} ± {custom_result['std_acc']:.4f}")


### 5. Generate Predictions for Best Configuration
Use this to generate predictions for Kaggle submission with the best individual feature configuration.


In [ ]:
# Generate predictions for best configuration
def generate_predictions(best_config, output_filename='predictions_best_config.csv'):
    """
    Generate predictions using the best individual feature configuration
    NOTE: This function trains on ALL available training data (no cross-validation)
    """
    print(f"\n=== Generating Predictions with Best Configuration ===")
    
    # Load test data
    inpTest = pd.read_csv('./../assignment2_files_2025/solutionInput.csv', sep=',', index_col='ID')
    
    # Use ALL training edges for final model (this is the correct approach for Kaggle)
    edges = np.array(list(G.edges()))
    pos_edges = edges
    rng = np.random.default_rng(seed)
    non_edges = np.array(list(nx.non_edges(G)))
    neg_edges = non_edges[rng.choice(len(non_edges), size=len(pos_edges), replace=False)]
    
    # Pre-compute metrics on the FULL training graph (this is correct for final submission)
    precomputed_metrics = precompute_metrics(G, COMMUNITY_CONFIG['resolution_small'], COMMUNITY_CONFIG['resolution_big'])
    
    # Generate features for training data
    X_train = []
    for (u, v) in np.vstack([pos_edges, neg_edges]):
        feat, _ = get_features(G, u, v, best_config, precomputed_metrics)
        X_train.append(feat)
    
    y_train = [1]*len(pos_edges) + [0]*len(neg_edges)
    
    # Train model
    clf = RandomForestClassifier(**MODEL_CONFIG)
    clf.fit(X_train, y_train)
    
    # Generate features for test data (using same precomputed metrics)
    test_features = []
    for _, row in inpTest.iterrows():
        feat, _ = get_features(G, int(row.iloc[0]), int(row.iloc[1]), best_config, precomputed_metrics)
        test_features.append(feat)
    
    # Generate predictions
    predictions = clf.predict(test_features)
    
    # Save predictions
    sub = pd.DataFrame({'ID': inpTest.index, 'prediction': predictions})
    sub.to_csv(output_filename, index=False)
    
    print(f"✅ Predictions saved to {output_filename}")
    return clf, test_features

### 6. Test Community Resolution Combinations
Explore different combinations of small and big community resolutions to find the optimal pair.


In [ ]:
# Test different community resolution combinations
print("=== Testing Community Resolution Combinations ===")

# # Test 1: Only community features with different resolutions
# community_only_results = test_community_resolutions(resolution_pairs=pairs)
# df_community_results = analyze_results(community_only_results)

# Test 2: Community features + some basic features
custom_config = {
    # === ATTRIBUTE FEATURES ===
    'attribute_i': False,              # Individual attribute value for node i
    'attribute_j': False,               # Individual attribute value for node j
    'attribute_equality': False,       # Disable attribute equality
    
    # === BASIC NODE FEATURES ===
    'degree_i': False,                  # Degree of node i
    'degree_j': False,                  # Degree of node j
    'clustering_i': False,             # Disable clustering for node i
    'clustering_j': False,             # Disable clustering for node j
    'pagerank_i': False,                # PageRank of node i
    'pagerank_j': False,                # PageRank of node j
    
    # === NEIGHBORHOOD FEATURES ===
    'common_neighbors': False,          # Number of common neighbors
    'jaccard_coefficient': False,       # Jaccard coefficient
    'adamic_adar': False,              # Disable Adamic-Adar
    'resource_allocation': False,        # Resource Allocation index
    'preferential_attachment': False,   # Preferential Attachment
    'salton_index': False,             # Disable Salton index
    'sorensen_index': False,             # Sorensen index
    'two_hop_neighbors': False,        # Disable 2-hop neighbors
    
    # === DERIVED FEATURES ===
    'degree_sum': False,                # Sum of degrees
    'degree_diff': False,               # Absolute difference of degrees
    'degree_product': False,             # Product of degrees
    'degree_ratio': False,              # Ratio of degrees
    'clustering_avg': False,           # Disable average clustering
    'pagerank_avg': False,             # Disable average PageRank
    
    # === COMMUNITY FEATURES ===
    'same_community': True,             # Same community indicator
    'community_size_i': False,           # Community size of node i
    'community_size_j': False,           # Community size of node j
    'community_size_diff': False,        # Absolute difference in community sizes
    'community_size_ratio': False,     # Disable community size ratio

    # === COMMUNITY FEATURES (Resolution 0.5 - Bigger Communities) ===
    'same_community_big': True,       # Same community indicator (big communities)
    'community_size_i_big': False,     # Community size of node i (big communities)
    'community_size_j_big': False,     # Community size of node j (big communities)
    'community_size_diff_big': False,  # Absolute difference in community sizes (big communities)
    'community_size_ratio_big': False, # Ratio of community sizes (big communities)
}

a = np.arange(0.6, 1.21, 0.2)
b = np.arange(1.5, 10.01, 0.5)

pairs = np.array(np.meshgrid(a, b)).T.reshape(-1, 2)

mixed_results = test_community_resolutions(feature_config=custom_config, resolution_pairs=pairs)
df_mixed_results = analyze_results(mixed_results)


=== Testing Community Resolution Combinations ===

=== Testing Community Resolution Combinations ===
Feature config: ['same_community', 'same_community_big']

--- Testing resolutions: small=0.6, big=1.5 ---

=== Testing Configuration ===
Enabled features: ['same_community', 'same_community_big']

--- Fold 1 ---
Fold 1 AUC: 0.8858, Accuracy: 0.8818

--- Fold 2 ---
Fold 2 AUC: 0.8936, Accuracy: 0.8807

--- Fold 3 ---
Fold 3 AUC: 0.8938, Accuracy: 0.8856

--- Fold 4 ---
Fold 4 AUC: 0.8945, Accuracy: 0.8860

--- Fold 5 ---
Fold 5 AUC: 0.8928, Accuracy: 0.8879

=== Results ===
Mean AUC: 0.8921 ± 0.0032
Mean Accuracy: 0.8844 ± 0.0027

--- Testing resolutions: small=0.6, big=2.0 ---

=== Testing Configuration ===
Enabled features: ['same_community', 'same_community_big']

--- Fold 1 ---
Fold 1 AUC: 0.8882, Accuracy: 0.8837

--- Fold 2 ---
Fold 2 AUC: 0.8932, Accuracy: 0.8792

--- Fold 3 ---
Fold 3 AUC: 0.8943, Accuracy: 0.8845

--- Fold 4 ---
Fold 4 AUC: 0.8945, Accuracy: 0.8845

--- Fold 5 -

In [ ]:
# Example: Generate predictions with current configuration
clf, test_features = generate_predictions(custom_config, 'predictions_with_relcomm.csv')

### 5. Data Leakage Prevention
**IMPORTANT**: Metrics must be computed only on training data to prevent data leakage. The framework automatically handles this correctly.


In [ ]:
# Example: Understanding data leakage prevention
print("=== Data Leakage Prevention Example ===")

# CORRECT: In cross-validation, metrics are computed only on training graph
print("✓ Cross-validation: Metrics computed on G_train (training edges only)")
print("✓ No data leakage: Validation edges never used for metric computation")

# CORRECT: For final submission, metrics computed on full training graph
print("✓ Final submission: Metrics computed on G (all training edges)")
print("✓ No data leakage: Test data never seen during training")

# Example: Extract features for a specific node pair (using full graph for demo)
node_i, node_j = 1, 2
features, feature_names = get_features(G, node_i, node_j, FEATURE_CONFIG)
print(f"Features for nodes ({node_i}, {node_j}): {len(features)} features")
print(f"Feature names: {feature_names[:5]}...")  # Show first 5 feature names

print("\n=== Key Points ===")
print("1. Cross-validation: Metrics computed on training fold only")
print("2. Final submission: Metrics computed on all training data")
print("3. Test data: Never used for metric computation")
print("4. Framework: Automatically prevents data leakage")


## Data Leakage Prevention

### The Problem
In link prediction, we must be careful about **data leakage**:
- **Training data**: Known edges used to train the model
- **Validation data**: Known edges used to evaluate the model (held out during training)
- **Test data**: Unknown edges we want to predict

### The Risk
If we compute graph metrics (PageRank, clustering, communities) on the **full graph** including validation/test edges, we're essentially using future information to predict the past.

### The Solution
1. **Cross-validation**: Compute metrics only on the training fold (`G_train`)
2. **Final submission**: Compute metrics on all training data (`G`) - this is correct because we're not evaluating anymore
3. **Test data**: Never used for metric computation

### How Our Framework Handles This
- `test_single_configuration()`: Uses `G_train` for each fold
- `generate_predictions()`: Uses `G` (all training data) for final model
- `precompute_metrics()`: Takes the appropriate graph as input
- `get_features()`: Uses precomputed metrics or computes on-the-fly

This ensures **no data leakage** while maintaining **good performance**.


## How to Use This Framework

### 1. **Individual Feature Control**
Each feature can be individually enabled/disabled by setting `True`/`False` in `FEATURE_CONFIG`:
- `attribute_i`: Individual attribute value for node i
- `degree_i`: Degree of node i
- `clustering_i`: Clustering coefficient of node i
- `common_neighbors`: Number of common neighbors
- `jaccard_coefficient`: Jaccard coefficient
- And many more...

### 2. **Individual Feature Testing**
Use `test_individual_features()` to test each feature separately and identify the most important ones.

### 3. **Feature Group Testing**
Use `test_feature_groups()` to test logical feature groups and their combinations.

### 4. **Custom Configuration**
Create your own configuration by modifying individual features in `FEATURE_CONFIG`.

### 5. **Best Configuration**
After finding the best individual features, use `generate_predictions()` to create your final submission.

### 6. **Feature Categories**
Features are organized into logical categories:
- **Attribute features**: Individual values and equality indicators
- **Basic node features**: Degree, clustering, PageRank for each node
- **Neighborhood features**: Common neighbors, similarity measures
- **Derived features**: Mathematical combinations of basic features
- **Community features (Small)**: Community-based indicators and sizes at resolution 1.0
- **Community features (Big)**: Community-based indicators and sizes at resolution 0.5

### 7. **Dual Community Features**
The framework now supports **two different community resolutions**:
- **Small communities** (resolution 1.0): More granular, smaller communities
- **Big communities** (resolution 0.5): Less granular, larger communities

This allows you to capture both local community structure (small) and global community structure (big) simultaneously.

### 8. **Community Resolution Testing**
- `test_community_resolutions()`: Test different resolution combinations
- `compare_community_features()`: Compare small vs big vs both community features
- Resolution pairs: Explore combinations like (0.3, 0.8), (0.5, 1.5), etc.

This granular control allows you to optimize performance by selecting only the most effective individual features and community resolution combinations.
